In [1]:
import rasterio as rio
import pandas as pd
import geopandas as gpd
from pathlib import Path
from datetime import datetime
import json
import os

In [2]:
s2_image_path = r"C:\Users\Kostas\Downloads\sentinel2_images_mean_2019-04-01_to_2019-05-01-0000000000-0000000000.tif"
envelopes_gdf = gpd.read_file(r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\PEP725\After_2016_sent_from_PEP725\pep725_outputs\PEP725_envelopes.geojson")

In [3]:
with rio.open(s2_image_path) as src:
    print(src.bounds)
    print(src.crs)
    src.close

BoundingBox(left=280320.0, bottom=5686440.0, right=695040.0, top=6101160.0)
EPSG:32632


In [3]:
envelopes_gdf.set_crs(32632, inplace=True, allow_override=True)
envelopes_gdf.head(10)

,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date,Label,geometry
0,5363,13.91670,54.0833,2,0,1050100,Alnus,Alnus glutinosa,60,2017,27,2017-01-27,DBL,"POLYGON ((817520.468 5999973.720, 825520.468 5..."
1,1554,7.51667,51.7333,60,72,2210500,Salix,Salix caprea,60,2017,29,2017-01-29,DBL,"POLYGON ((393567.206 5728416.903, 401567.206 5..."
2,3120,8.68333,49.5500,140,261,1050100,Alnus,Alnus glutinosa,60,2017,31,2017-01-31,DBL,"POLYGON ((473094.080 5484647.767, 481094.080 5..."
3,2021,8.58333,50.0000,100,101,1050100,Alnus,Alnus glutinosa,60,2017,32,2017-02-01,DBL,"POLYGON ((466138.525 5534713.881, 474138.525 5..."
4,1521,7.83333,51.7000,60,58,1050100,Alnus,Alnus glutinosa,60,2017,33,2017-02-02,DBL,"POLYGON ((415374.473 5724316.443, 423374.473 5..."
5,1710,8.71667,51.7333,100,99,1050100,Alnus,Alnus glutinosa,60,2017,34,2017-02-03,DBL,"POLYGON ((476433.883 5727413.739, 484433.883 5..."
6,1234,6.60000,51.3833,35,34,1050100,Alnus,Alnus glutinosa,60,2017,36,2017-02-05,DBL,"POLYGON ((328995.067 5691184.337, 336995.067 5..."
7,1111,10.85000,52.4333,60,57,1050100,Alnus,Alnus glutinosa,60,2017,37,2017-02-06,DBL,"POLYGON ((621768.054 5806842.511, 629768.054 5..."
8,8154,6.66667,51.7667,40,38,1050100,Alnus,Alnus glutinosa,60,2017,37,2017-02-06,DBL,"POLYGON ((334994.639 5733666.257, 342994.639 5..."
9,494,10.20000,51.5333,200,175,1050100,Alnus,Alnus glutinosa,60,2017,39,2017-02-08,DBL,"POLYGON ((579233.202 5705815.075, 587233.202 5..."


In [ ]:
# Separate the gdfs by year
#envelopes_gdf_2019 = envelopes_gdf[envelopes_gdf['year'] == 2019]
#envelopes_gdf_2020 = envelopes_gdf[envelopes_gdf['year'] == 2020]

In [ ]:
#envelopes_gdf_2019.head()

## 1. Temporal filter

Now that everything is loaded the temporal filter should be applied

In [4]:
# Function to extract the dates from the filename of GEE S2 images

def imageNamingGEEfiles(raster_path):
    # Example file name: sentinel2_images_mean_2019-07-01_to_2019-08-01-0000006912-0000006912.tif
    string_parts = raster_path.split("_")
    start_date = string_parts[3]
    token = string_parts[5]
    token_string_parts = token.split("-")
    end_date = token_string_parts[0] + "-" + token_string_parts[1] + "-" + token_string_parts[2]
    # Save the month and year to variables
    s2month = datetime.strptime(start_date, '%Y-%m-%d').month
    s2year = datetime.strptime(start_date, '%Y-%m-%d').year
    return start_date, end_date, s2month, s2year

a, b, c, d = imageNamingGEEfiles("sentinel2_images_mean_2019-07-01_to_2019-08-01-0000006912-0000006912.tif")
print(a, b, c, d)

2019-07-01 2019-08-01 7 2019


In [ ]:
# Pathlib stuff: Path methods: anchor, parent, name, stem, suffixes
# This can be used for easier extraction of dates from the filename
print("anchor: ", Path(s2_image_path).anchor)
print("parent: ", Path(s2_image_path).parent)
print("name: ", Path(s2_image_path).name)
print("stem: ", Path(s2_image_path).stem)
print("suffixes: ", Path(s2_image_path).suffixes)
print("Normal print: ", s2_image_path)


In [7]:
# Get the start and end dates of the image from its name
s2_image_start_date, s2_image_end_date, s2month, s2year = imageNamingGEEfiles(Path(s2_image_path).name)
print(s2_image_start_date, s2_image_end_date, s2month, s2year)

2019-04-01 2019-05-01 4 2019


__________

Datetime stuff in order to do date comparisons and find all the dates that are represented in an image

In [5]:
# Converting the date column to datetime data type

envelopes_gdf['date'] = pd.to_datetime(envelopes_gdf['date'], format='%Y-%m-%d').dt.date

In [8]:
# Converting the outputs to datetime.date dtype
s2_image_start_date = datetime.strptime(s2_image_start_date, '%Y-%m-%d').date()
s2_image_end_date = datetime.strptime(s2_image_end_date, '%Y-%m-%d').date()


In [9]:
# Creating a mask to filter the dates that are needed
temporal_mask = (envelopes_gdf.date > s2_image_start_date) & (envelopes_gdf.date < s2_image_end_date)
display(envelopes_gdf.loc[temporal_mask])
s2_image_gdf = envelopes_gdf.loc[temporal_mask]


,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date,Label,geometry
49410,110,10.48330,53.5000,40,54,1030100,Aesculus,Aesculus hippocastanum,11,2019,92,2019-04-02,DBL,"POLYGON ((594387.807 5924917.608, 602387.807 5..."
49411,148,10.83330,54.2500,55,35,1180100,Betula,Betula pendula,11,2019,92,2019-04-02,DBL,"POLYGON ((615444.341 6008888.454, 623444.341 6..."
49412,299,10.25000,53.5167,35,41,1580100,Fraxinus,Fraxinus excelsior,60,2019,92,2019-04-02,DBL,"POLYGON ((578881.130 5926478.618, 586881.130 5..."
49413,354,8.50000,52.7000,35,41,2290100,Sorbus,Sorbus aucuparia,11,2019,92,2019-04-02,DBL,"POLYGON ((462212.807 5835016.122, 470212.807 5..."
49414,370,8.58333,52.5500,40,40,1050100,Alnus,Alnus glutinosa,11,2019,92,2019-04-02,DBL,"POLYGON ((467747.325 5818295.203, 475747.325 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123541,8212,13.88330,51.1333,250,236,10000,perm_grass,None,111,2019,120,2019-04-30,M,"POLYGON ((837588.487 5671994.839, 845588.487 5..."
123542,8227,7.26667,53.0500,3,4,10000,perm_grass,None,111,2019,120,2019-04-30,M,"POLYGON ((379814.909 5875237.296, 387814.909 5..."
123543,20320,8.63333,52.1500,110,82,10000,perm_grass,None,111,2019,120,2019-04-30,M,"POLYGON ((470912.168 5773785.241, 478912.168 5..."
123544,20637,6.53333,51.7333,22,24,10000,perm_grass,None,111,2019,120,2019-04-30,M,"POLYGON ((325669.511 5730255.241, 333669.511 5..."


## Target extraction

In [10]:
def addMonths(gdf):
    # Convert the date to datetime type to work later
    gdf['date'] = pd.to_datetime(gdf['date'])

    # Create a Series with the month (1-12)
    # It finds the month (int 1-12) based on the .month method of the datetime property
    # It achieves that by mapping a lambda function on each element of the date column. Therefore the result is just the month number

    getmonth = gdf['date'].map(lambda x:x.month)

    # Merge this into the gdf
    gdf = gdf.merge(getmonth, left_index=True, right_index=True)

    # Rename the column
    gdf.rename(columns = {'date_y':'month'}, inplace = True)
    return gdf

In [11]:
envelopes_gdf = addMonths(envelopes_gdf)

In [12]:
envelopes_gdf.tail()

,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date_x,Label,geometry,month
129324,5456,13.75000,50.7333,875,864,10000,perm_grass,None,131,2021,244,2021-09-01,M,"POLYGON ((831136.405 5626933.921, 839136.405 5...",9
129325,961,7.50000,52.7667,30,33,10000,perm_grass,None,111,2021,257,2021-09-14,M,"POLYGON ((394795.749 5843373.189, 402795.749 5...",9
129326,20595,8.50000,49.4667,95,86,10000,perm_grass,None,131,2021,258,2021-09-15,M,"POLYGON ((459771.720 5475458.856, 467771.720 5...",9
129327,19312,8.36667,48.6000,490,622,10000,perm_grass,None,131,2021,285,2021-10-12,M,"POLYGON ((449306.954 5379184.811, 457306.954 5...",10
129328,8197,10.98330,48.2333,525,504,10000,perm_grass,None,111,2021,289,2021-10-16,M,"POLYGON ((643273.716 5340132.606, 651273.716 5...",10


In [13]:
# Group observations by s_id and month, and calculate the label with the maximum frequency for each group
freqresults_df = envelopes_gdf.groupby(['s_id', pd.Grouper(key='month'), pd.Grouper(key='year')])['Label'].apply(lambda x: x.value_counts().index[0]).reset_index()

# Rename the column with the label
freqresults_df = freqresults_df.rename(columns={'Label': 'max_label'})


In [16]:
# Check if it works correctly
freqresults_df.sort_values(['year', 'month']).head(50)

,s_id,month,year,max_label
6853,1554,1,2017,DBL
15589,3120,1,2017,DBL
27370,5363,1,2017,DBL
35846,19544,1,2017,M
0,21,2,2017,DBL
96,43,2,2017,DBL
106,66,2,2017,DBL
120,94,2,2017,DBL
162,106,2,2017,DBL
366,148,2,2017,DBL


In [15]:
freqresults_df['max_label'].unique()

array(['DBL', 'M', 'EC'], dtype=object)

In [17]:
freqresults_df['max_label'].value_counts()

DBL    37914
M       1744
EC        16
Name: max_label, dtype: int64

In [16]:
freqresults_df[freqresults_df['max_label'] == 'M'].head(10)

,s_id,month,year,max_label
18,21,6,2017,M
19,21,6,2018,M
20,21,6,2019,M
59,32,6,2017,M
60,32,6,2018,M
62,32,6,2020,M
113,66,7,2017,M
174,110,3,2021,M
186,110,6,2018,M
187,110,6,2019,M


___________

## Spatial filter

In [18]:
# Convert the envelopes_gdf to a list to work with the functions
s2_image_gdf_list = s2_image_gdf.geometry.tolist()

# This is used to save the indices and then extract the targets directly from the gdf
s2_image_gdf_index = s2_image_gdf.index.tolist()


In [19]:
output_dir = Path(r'C:\Users\Kostas\Desktop\GIMA\Module_7\Data\filtered_patches_GEE')

In [20]:
"""Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""

def getFeatures(gdf):
        return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [21]:
'''
This function reads the envelope list and a raster, checks if the polygons are fully contained in the raster 
and returns 5 lists, 4 with the boundary coordinates for all the envelopes that are fully contained in the raster 
and one of their indexes from the full_index_list.
'''

def getContainedEnvelopeCoords (raster, envelope_list, full_index_list):
    with rio.open(raster, driver='GTiff') as src:
        raster_extent = src.bounds
        
        # List initialization
        minx_list = []
        miny_list = []
        maxx_list = []
        maxy_list = []
        index_list = []
        for i in range(0, len(envelope_list)):
            poly_extent = envelope_list[i].bounds

            # Check if the polygon is fully inside the raster's extent
            if (poly_extent[0] >= raster_extent[0] and poly_extent[2] <= raster_extent[2] and
                poly_extent[1] >= raster_extent[1] and poly_extent[3] <= raster_extent[3]):
                    minx_list.append(poly_extent[0])
                    miny_list.append(poly_extent[1])
                    maxx_list.append(poly_extent[2])
                    maxy_list.append(poly_extent[3])
                    index_list.append(full_index_list[i])
    return minx_list, miny_list, maxx_list, maxy_list, index_list

In [22]:
for poly in s2_image_gdf_list:
    print(poly)
    break

for i in range(0, len(s2_image_gdf_list)):
    print(i)
    break


POLYGON ((594387.8070813899 5924917.6078205975, 602387.8070813899 5924917.6078205975, 602387.8070813899 5932917.6078205975, 594387.8070813899 5932917.6078205975, 594387.8070813899 5924917.6078205975))
0


In [23]:
'''
This function receives a raster file (.tif) and the boundary coordinates for a polygon. 
It then clips the raster to the extent of the polygon. 
The polygon has to intersect the raster for the operation to be completed
'''

from shapely.geometry import box
from rasterio.mask import mask

def exportImage(raster, output_path, minx, miny, maxx, maxy):
    # open the raster file (Single Band)
    data = rio.open(raster, driver='GTiff')

    # Create a bounding box from the polygon min-max coordinates    
    bbox = box(minx, miny, maxx, maxy)
    # Create a geodataframe with a single polygon so that it can be used with rasterio
    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs='32632')
    # Transform the geodataframe to a GeoJSON-like object that can be used as an input in the rasterio mask function
    coords = getFeatures(geo)
    #print(coords)
    
    # Mask and crop the raster AOI where polygon overlaps the whole raster
    out_img, out_transform = mask(data, shapes=coords, crop=True)
    # Define resolution and more
    out_profile = data.profile.copy()
    
    out_profile.update({'driver':'GTiff', 'width': out_img.shape[2],'height': out_img.shape[1], 'transform': out_transform})
    
    # Write the extracted raster patch to a file
    with rio.open(output_path, 'w', **out_profile) as dst:
        dst.write(out_img)
    
    # data.close()
    # data = None

In [24]:
minx_list, miny_list, maxx_list, maxy_list, index_list = getContainedEnvelopeCoords(s2_image_path, s2_image_gdf_list, s2_image_gdf_index)

In [25]:
# Test to check if the index list works. It works.

for test in range(0, 10):
    print(minx_list[test], miny_list[test], maxx_list[test], maxy_list[test], index_list[test])
    print("---------")

print("Check if the elements on the index and coord lists are the ones from the actual gdf")
s2_image_gdf.loc[49419]

594387.8070813899 5924917.6078205975 602387.8070813899 5932917.6078205975 49410
---------
615444.3410472146 6008888.454367091 623444.3410472146 6016888.454367091 49411
---------
578881.1304628432 5926478.618106418 586881.1304628432 5934478.618106418 49412
---------
462212.8066672371 5835016.122399375 470212.8066672371 5843016.122399375 49413
---------
467747.3246259597 5818295.20291999 475747.3246259597 5826295.20291999 49414
---------
546005.9353980366 5790659.128762336 554005.9353980366 5798659.128762336 49415
---------
546005.9353980366 5790659.128762336 554005.9353980366 5798659.128762336 49416
---------
530095.2360452933 5790523.630913248 538095.2360452933 5798523.630913248 49417
---------
526616.562715307 5801624.127417376 534616.562715307 5809624.127417376 49418
---------
523183.964558654 5807165.645333859 531183.964558654 5815165.645333859 49419
---------
Check if the elements on the index and coord lists are the ones from the actual gdf


s_id                                                      460
lon                                                       9.4
lat                                                     52.45
alt                                                        45
alt_dem                                                    46
gss_id                                                1580100
genus                                                Fraxinus
species                                    Fraxinus excelsior
phase_id                                                   60
year                                                     2019
day                                                        92
date                                               2019-04-02
Label                                                     DBL
geometry    POLYGON ((523183.964558654 5807165.645333859, ...
Name: 49419, dtype: object

In [26]:
s2_image_gdf.loc[49410, 's_id']

110

In [115]:
s_id_list = envelopes_gdf.loc[index_list, 's_id'].tolist()

In [35]:
# New try with s_id list. This list has everything in order. The one with envelopes_gdf.loc[index_list, 's_id'].tolist() changes the sequence.
s_id_list = envelopes_gdf['s_id'].tolist()

In [34]:
envelopes_gdf.head(10)

,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date_x,Label,geometry,month
0,5363,13.91670,54.0833,2,0,1050100,Alnus,Alnus glutinosa,60,2017,27,2017-01-27,DBL,"POLYGON ((817520.468 5999973.720, 825520.468 5...",1
1,1554,7.51667,51.7333,60,72,2210500,Salix,Salix caprea,60,2017,29,2017-01-29,DBL,"POLYGON ((393567.206 5728416.903, 401567.206 5...",1
2,3120,8.68333,49.5500,140,261,1050100,Alnus,Alnus glutinosa,60,2017,31,2017-01-31,DBL,"POLYGON ((473094.080 5484647.767, 481094.080 5...",1
3,2021,8.58333,50.0000,100,101,1050100,Alnus,Alnus glutinosa,60,2017,32,2017-02-01,DBL,"POLYGON ((466138.525 5534713.881, 474138.525 5...",2
4,1521,7.83333,51.7000,60,58,1050100,Alnus,Alnus glutinosa,60,2017,33,2017-02-02,DBL,"POLYGON ((415374.473 5724316.443, 423374.473 5...",2
5,1710,8.71667,51.7333,100,99,1050100,Alnus,Alnus glutinosa,60,2017,34,2017-02-03,DBL,"POLYGON ((476433.883 5727413.739, 484433.883 5...",2
6,1234,6.60000,51.3833,35,34,1050100,Alnus,Alnus glutinosa,60,2017,36,2017-02-05,DBL,"POLYGON ((328995.067 5691184.337, 336995.067 5...",2
7,1111,10.85000,52.4333,60,57,1050100,Alnus,Alnus glutinosa,60,2017,37,2017-02-06,DBL,"POLYGON ((621768.054 5806842.511, 629768.054 5...",2
8,8154,6.66667,51.7667,40,38,1050100,Alnus,Alnus glutinosa,60,2017,37,2017-02-06,DBL,"POLYGON ((334994.639 5733666.257, 342994.639 5...",2
9,494,10.20000,51.5333,200,175,1050100,Alnus,Alnus glutinosa,60,2017,39,2017-02-08,DBL,"POLYGON ((579233.202 5705815.075, 587233.202 5...",2


In [53]:
i = 0
while i <= 10:
    print(s_id_list[i])
    i += 1

110
148
299
354
370
411
411
415
427
460
461


In [37]:
freqresults_df.head()

,s_id,month,year,max_label
0,21,2,2017,DBL
1,21,2,2020,DBL
2,21,2,2021,DBL
3,21,3,2017,DBL
4,21,3,2018,DBL


In [54]:
# Test with freqresult filter
f = {'s_id': [5363, 1554, 3120, 2021, 1521], 'month': [2, 4, 4, 4, 4], 'year': [2019, 2019, 2019, 2019, 2020], 'max_label': ['DBL', 'EC', 'M', 'DBL','DBL']}
freqtest_df = pd.DataFrame(data=f)
freqtest_df.head()

,s_id,month,year,max_label
0,5363,2,2019,DBL
1,1554,4,2019,EC
2,3120,4,2019,M
3,2021,4,2019,DBL
4,1521,4,2020,DBL


In [39]:
s_id_list_test = [10, 20, 30, 40, 5363, 2, 1554, 12, 4, 3120, 22, 2021, 65, 1521]

In [57]:
result_list = []
print(f'Getting all the maxfreq labels for month {s2month} and year {s2year}')
for i in range(0, len(s_id_list_test)):
    result = freqtest_df.loc[(freqtest_df['s_id'].isin(s_id_list_test)) & (freqtest_df['year'] == s2year) & (freqtest_df['month'] == s2month), 'max_label']

    result_list.append(result)

Getting all the maxfreq labels for month 4 and year 2019


In [82]:
#freqtest_df[freqtest_df['s_id'].isin(s_id_list_test)]
freqtest_df[(freqtest_df['s_id'].isin(s_id_list_test)) & (freqtest_df['year'] == s2year) & (freqtest_df['month'] == s2month)]


,s_id,month,year,max_label
1,1554,4,2019,EC
2,3120,4,2019,M
3,2021,4,2019,DBL


In [98]:
freqresults_df['max_label'].value_counts()

DBL    37914
M       1744
EC        16
Name: max_label, dtype: int64

In [97]:
freqresults_df[freqresults_df['max_label'] == 'EC']

,s_id,month,year,max_label
6679,1521,5,2019,EC
12512,2566,4,2017,EC
15070,3035,5,2018,EC
16563,3268,5,2020,EC
18187,3603,4,2021,EC
21212,4045,3,2018,EC
24479,4672,5,2020,EC
27967,5456,6,2021,EC
28113,5478,5,2021,EC
30013,5781,5,2019,EC


In [95]:
# I think that this works great! Maybe not. I see only DBL
freqresults_df[(freqresults_df['s_id'].isin(s_id_list_test)) & (freqresults_df['year'] == s2year) & (freqresults_df['month'] == s2month)]

,s_id,month,year,max_label
6675,1521,4,2019,DBL
9317,2021,4,2019,DBL
15601,3120,4,2019,DBL
27383,5363,4,2019,DBL


In [94]:
print(result_list[0])

1     EC
2      M
3    DBL
Name: max_label, dtype: object


In [48]:
for x in result_list:
    print(x)

Series([], Name: max_label, dtype: object)
Series([], Name: max_label, dtype: object)
Series([], Name: max_label, dtype: object)
Series([], Name: max_label, dtype: object)
Series([], Name: max_label, dtype: object)
Series([], Name: max_label, dtype: object)
1    EC
Name: max_label, dtype: object
Series([], Name: max_label, dtype: object)
Series([], Name: max_label, dtype: object)
2    M
Name: max_label, dtype: object
Series([], Name: max_label, dtype: object)
3    DBL
Name: max_label, dtype: object
Series([], Name: max_label, dtype: object)
Series([], Name: max_label, dtype: object)


In [57]:
def unique(list1):
    x = np.array(list1)
    print(np.unique(x))

unique(result_list)

['EC']


In [ ]:
print("Creating patches for the image: ", Path(s2_image_path).name)

# Iterating over each envelope in the gdf

for i in range(0, len(minx_list)):
    # Get the station id for the station with the index in the i-th position
    station_id = s2_image_gdf.loc[index_list[i], 's_id']
    output_name = os.path.join(output_dir, Path(s2_image_path).stem + f'_station_{station_id}_index_{index_list[i]}.tif')
    print(f"\t Patch {i+1} out of {len(minx_list) + 1}")
    exportImage(s2_image_path, output_name, minx_list[i], miny_list[i], maxx_list[i], maxy_list[i])
print('Patch creation completed!') 



    

In [128]:
envelopes_gdf[(envelopes_gdf['year'] == 2019) & (envelopes_gdf['month'] == 4) & (envelopes_gdf['Label'] == 'EC')]

,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date_x,Label,geometry,month
115337,3663,12.55000,48.0333,500,545,2030100,Picea,Picea abies,10,2019,91,2019-04-01,EC,"POLYGON ((760625.803 5322100.595, 768625.803 5...",4
115338,3849,11.81670,49.6167,400,386,2030100,Picea,Picea abies,10,2019,91,2019-04-01,EC,"POLYGON ((699451.924 5495825.826, 707451.924 5...",4
115339,6178,10.91670,50.4333,430,445,2030100,Picea,Picea abies,10,2019,94,2019-04-04,EC,"POLYGON ((632120.990 5584564.238, 640120.990 5...",4
115340,21405,11.81670,53.5833,55,56,2030100,Picea,Picea abies,10,2019,95,2019-04-05,EC,"POLYGON ((682450.224 5936850.354, 690450.224 5...",4
115341,21,9.81667,54.4333,25,19,2030100,Picea,Picea abies,10,2019,96,2019-04-06,EC,"POLYGON ((548974.306 6028039.686, 556974.306 6...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115809,19315,10.21670,50.3000,250,245,2030100,Picea,Picea abies,10,2019,120,2019-04-30,EC,"POLYGON ((582652.335 5568694.875, 590652.335 5...",4
115810,19535,9.71667,48.6500,430,446,2030100,Picea,Picea abies,10,2019,120,2019-04-30,EC,"POLYGON ((548785.185 5384796.956, 556785.185 5...",4
115811,20564,9.80000,52.4500,50,40,2030100,Picea,Picea abies,10,2019,120,2019-04-30,EC,"POLYGON ((550367.590 5807391.344, 558367.590 5...",4
115812,21402,10.95000,50.2500,291,352,2030100,Picea,Picea abies,10,2019,120,2019-04-30,EC,"POLYGON ((635019.904 5564246.614, 643019.904 5...",4
